1. [Backprop guide: a cleaner numpy implementation](#10)
2. [Building the classes](#20)
	1. [Simplest layers](#21)
	2. [Layers that learn](#22)
	3. [The network itself, as a class](#23)
	4. [Setting up "cool" layers: ReLUs, noise, dropout, etc.](#24)
3. [Testing the network on real data](#30)
	1. [First, testing that things work](#31)
	2. [Gradient checking](#32)
	3. [Testing the model on the mini-MNIST dataset](#33)
	4. [Testing the model on the full MNIST dataset](#34)

# Backprop guide: a cleaner numpy implementation <a name="10"></a>

I've already made a simple feed forward neural network using Numpy; however, it's a lot cleaner to use a object-oriented design. I'll work that out here.

I will define the various neural network layers as classes. I'll use inheritence here, but it's not really crucial. I just want to make sure that all the layer classes follow some sort of common style and logic.

In [1]:
import numpy as np
import matplotlib.pyplot as plt # For plotting things
from sklearn.datasets import load_digits # Data to be used

# Building the classes <a name="20"></a>

## Simplest layers <a name="21"></a>

Let's start with an identify layer. You can think of as a sigmoid or a ReLU activation function layer, except that the identity layer does nothing at all, $f(x) = y$.

In [2]:
class IdentityActivation:
    def __init__(self, predecessor):
        # Remember what precedes this layer
        self.predecessor = predecessor
        # The activation function keeps the dimensions of its predecessor
        self.input_size = self.predecessor.output_size        
        self.output_size = self.input_size
        # Create an empty matrix to store this layer's last activation
        self.activation = np.zeros(self.output_size)
        # Initialize weights, if necessary
        self.init_weights()
        # Will never require a gradient
        self.require_gradient = False
    # This activation function has no parameters, so pass
    def init_weights(self):
        pass
    # It also has to gradients, so pass here too
    def zero_grad(self):
        pass
    # During a forward pass, it just passes its input forward unmodified
    def forward(self, x, evaluate=False):
        # Save a copy of the activation
        self.activation = x
        return self.activation
    # During backrop it passes the delta on unmodified
    def backprop(self, delta, y):
        return delta
    # It has no parameters
    def report_params(self):
        return []

The above doesn't really do anything except let me develop a blueprint for the rest of my classes. I definitely need `forward`, `backprop`, and `update` methods for all my layers. It's also nice to have some `identify` method to make debugging easier.

Actually, here are all the requirements:
* `params()` method that dumps its parameters to the console
* `name()` method for dumping its properties to the console
* `init_weights()` when you want its parameters initialized
* `zero_grad()` when you want to zero its saved gradients
* `forward()` when you want it to estimate input
* `backprop()` when you want it to backpropagate error
* `report_params()` return a list of all of its parameters

What is not included
* `update()` when you want it to update its parameters: this should be done by an optimizer since you want to choose between SGD, adagrad, Adam, etc.

It's nice to have all of these as manual methods because you can start taking apart the NN to do a few things. For example, if I want to do a gradient check, I just want to calculate the gradients without actually updating them. When I'm done checking, I can call `zero_grad` and nothing will have changed.

The Input layer is nothing really special. It just has to take a size (of features, variables), and otherwise it just feeds data to the first hidden layer, so it turns out it's pretty much just an identity activation.

In [3]:
class InputLayer(IdentityActivation):
    def __init__(self, input_size):
        # The size here is determined by the data that's going to be used
        self.input_size = (0, input_size)
        self.output_size = self.input_size
        # Create an empty matrix to store this layer's last activation
        self.activation = np.zeros(self.output_size)

Let's try our hand at a sigmoid layer.

In [4]:
class SigmoidActivation(IdentityActivation):
    # During a forward pass, it just applies the sigmoid function
    def forward(self, x, evaluate=False):
        self.activation = 1.0/(1.0+np.exp(-x))
        return self.activation
    # During backprop, it passes the delta through its derivative
    def backprop(self, delta, y):
        return delta * self.activation * (1 - self.activation)

## Layers that learn <a name="22"></a>

Let's try a dense layer next.

I am putting in two useful things here. The `use_bias` flag is pretty easy as a way of disabling the bias term, which is sometimes necessary. The `require_gradient` can be useful in some situations, such as freezing a layer.

One way of dealing with parameters is with a `Variable` class. This is how torch and pytorch do it: I'll keep the same names since there's no point messing with what works.

We need to do operator overloading. [You can find a list of the operators you can define here.](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types) *However, you cannot override the assignment operator `=`.*

In [5]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.shape = self.data.shape
        self.grad = np.zeros(self.shape)
    # Cheap way of zeroing its gradient
    def zero_grad(self):
        self.grad *= 0

In [6]:
class DenseLayer(IdentityActivation):
    def __init__(self, predecessor, hidden, use_bias=True, require_gradient=True, positive_params=False):
        # Remember what precedes this layer
        self.predecessor = predecessor
        self.input_size = self.predecessor.output_size
        self.hidden = hidden
        self.output_size = (0, self.hidden)
        # It is possible that we don't want a bias term
        self.use_bias = use_bias
        # If you need non-negative parameters
        self.positive_params = positive_params
        # Save its activation
        self.activation = np.zeros(self.output_size)
        # Initialize the weights and biases
        self.init_params()
        # Most of the time, this layer will use gradients to train itself
        # However, this can be disabled manually
        self.require_gradient = require_gradient
    def zero_grad(self):
        self.weight.zero_grad()
        if self.use_bias:
            self.bias.zero_grad()
    def init_params(self):
        size_measure = self.input_size[1]
        if self.positive_params:
            lower, upper = 0., 0.5
        else:
            lower, upper = -1., 1.
        # Weights are initialized by a normal distribution
        self.weight = Variable(
            np.sqrt(2/size_measure) * np.random.uniform(lower, upper, size=(self.input_size[1], self.hidden))
        )
        if self.use_bias:
            self.bias = Variable(
                np.sqrt(2/size_measure) * np.random.uniform(lower, upper, size=(1, self.hidden))
            )
    # The forward pass is a matrix multiplication, with optional bias
    def forward(self, x, evaluate=False):
        x = x @ self.weight.data
        if self.use_bias:
            x += self.bias.data
        self.activation = x
        return self.activation
    # The delta just needs to be multipled by the layer's weight
    def backprop(self, delta, y):
        # Only calculate gradients if it's required
        if self.require_gradient:
            # The weight update requires the previous layer's activation
            self.weight.grad += self.predecessor.activation.transpose() @ delta
            # The bias update requires the delta to be "squished"
            # This can be done by multiplying by a vector of 1s
            if self.use_bias:
                self.bias.grad += np.ones((1, delta.shape[0])) @ delta
        return delta @ self.weight.data.transpose()
    # This DenseLayer is the first example of a layer with parameters
    def report_params(self):
        if self.use_bias:
            return [self.bias, self.weight]
        else:
            return [self.weight]

Alright... Now we must attempt an output layer, particulary a **softmax output activation** combined with a **dense layer**. We combine these because the math collapses the two during backprop, making things more efficient.

This will be a special layer because it will deliver predictions during feed forward and also trigger the start of backprop.

We want to run feed forward and backprop with for-loops. For feed forward, it will just have to output the proper `x`, which is not overly difficult. For backprop, it will require its own activations (easy) and the true outputs `y` (which makes things a bit clunky).

This will be an opportunity to use some nice inheritence.

In [7]:
class SigmoidNLL(DenseLayer):
    # The forward pass is a matrix multiplication, with optional bias
    def forward(self, x, evaluate=False):
        # The feed forward starts off normal
        x = x @ self.weight.data
        if self.use_bias:
            x += self.bias.data
        # It changes when we apply the sigmoid
        self.activation = 1.0/(1.0+np.exp(-x))
        return self.activation
    # The delta is started here
    def backprop(self, delta, y):
        # Starting the delta
        delta = self.activation - y
        # The update is the same as a DenseLayer
        self.weight.grad += self.predecessor.activation.transpose() @ delta
        if self.use_bias:
            self.bias.grad += np.ones((1, delta.shape[0])) @ delta
        # The delta is passed backwards like a Denselayer
        return delta @ self.weight.data.transpose()

class SoftmaxCrossEntropy(DenseLayer):
    # The forward pass is a matrix multiplication, with optional bias
    def forward(self, x, evaluate=False):
        # The feed forward starts off normal
        x = x @ self.weight.data
        if self.use_bias:
            x += self.bias.data
        # It changes when we apply the sigmoid
        softmax_sum = np.exp(x) @ np.ones((self.hidden, 1))
        self.activation = np.exp(x) / softmax_sum
        return self.activation
    # The delta is started here
    def backprop(self, delta, y):
        # Starting the delta
        delta = self.activation - y
        # The update is the same as a DenseLayer
        self.weight.grad += self.predecessor.activation.transpose() @ delta
        if self.use_bias:
            self.bias.grad += np.ones((1, delta.shape[0])) @ delta
        # The delta is passed backwards like a Denselayer
        return delta @ self.weight.data.transpose()

## The network itself, as a class <a name="23"></a>

I will set up a NeuralNetwork class very, very simplistically. It will just be a list that you append to. Nevertheless, I prefer this way because it forces you to be verbose when you're stacking your layers.

The `FeedForward` method is very simple though.

In [8]:
# This class stores a list of layers for training
class NeuralNetwork:
    def __init__(self):
        self.layers = []
    def feed_forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    # When you want the model to know you're not training
    def evaluate(self, x):
        for layer in self.layers:
            x = layer.forward(x, evaluate=True)
        return x
    def back_propagation(self, y):
        delta = np.zeros((0,0))
        for layer in reversed(self.layers):
            delta = layer.backprop(delta, y)
    def step(self,lr):
        for layer in self.layers:
            layer.update(lr)
    def zero_gradients(self):
        for layer in self.layers:
            layer.zero_grad()

For this network to work, it will require an optimizer.

In [9]:
class SGDOptimizer:
    def __init__(self, list_of_layers):
        self.list_of_layers = list_of_layers
        self.list_of_variables = []
        for layer in self.list_of_layers:
            self.list_of_variables += layer.report_params()
    def step(self, lr):
        for variable in self.list_of_variables:
            variable.data -= lr * variable.grad

class AdaGradOptimizer:
    def __init__(self, list_of_layers):
        self.list_of_layers = list_of_layers
        self.list_of_variables = []
        for layer in self.list_of_layers:
            self.list_of_variables += layer.report_params()
        self.gradient_histories = dict()
        for variable in self.list_of_variables:
            self.gradient_histories[variable] = np.ones(variable.shape)
    def step(self, lr):
        for variable in self.list_of_variables:
            variable.data -= (lr / np.sqrt(self.gradient_histories[variable])) * variable.grad
            self.gradient_histories[variable] += variable.grad**2

class AdamOptimizer:
    def __init__(self, list_of_layers, beta1=0.9, beta2=0.999, eps=0.00000001):
        self.list_of_layers = list_of_layers
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.list_of_variables = []
        for layer in self.list_of_layers:
            self.list_of_variables += layer.report_params()
        self.adam_mean = dict()
        self.adam_var = dict()
        for variable in self.list_of_variables:
            self.adam_mean[variable] = np.zeros(variable.shape)
            self.adam_var[variable] = np.zeros(variable.shape)
    def step(self, lr):
        for variable in self.list_of_variables:
            self.adam_mean[variable] = self.adam_mean[variable] * self.beta1 + variable.grad * (1 - self.beta1)
            self.adam_var[variable] = self.adam_var[variable] * self.beta2 + (variable.grad**2) * (1 - self.beta2)
            mean_hat = self.adam_mean[variable] / (1 - self.beta1)
            var_hat = self.adam_var[variable] / (1 - self.beta2)
            variable.data -= (lr * mean_hat)/(np.sqrt(var_hat) + self.eps)

## Setting up "cool" layers: ReLUs, noise, dropout, etc. <a name="24"></a>

Lets set up a ReLU. https://stackoverflow.com/questions/32109319/how-to-implement-the-relu-function-in-numpy

I also want a softplus layer. Because I like them. https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

In [10]:
class ReLUActivation(IdentityActivation):
    # During a forward pass, it just applies the sigmoid function
    def forward(self, x, evaluate=False):
        self.activation = x * (x > 0)
        return self.activation
    # During backprop, it passes the delta through its derivative
    def backprop(self, delta, y):
        return delta * (self.activation > 0)

class SoftplusActivation(IdentityActivation):
    # During a forward pass, it just applies the sigmoid function
    def forward(self, x, evaluate=False):
        self.activation = np.log(1.0 + np.exp(x))
        return self.activation
    # During backprop, it passes the delta through its derivative
    def backprop(self, delta, y):
        return delta * 1.0/(1.0 + np.exp(-self.activation))

Let's also set up a noise-generating layer

In [11]:
class GaussianNoise(IdentityActivation):
    def __init__(self, predecessor, mean=0, stdev=0.1):
        self.predecessor = predecessor
        self.input_size = self.predecessor.output_size        
        self.output_size = self.input_size
        self.activation = np.zeros(self.output_size)
        self.gradient = np.zeros(self.output_size)
        self.init_weights()
        self.require_gradient = False
        # Noise
        self.mean = mean
        self.stdev=stdev
    def forward(self, x, evaluate=False):
        if evaluate:
            self.activation = x
        else:
            noise = np.random.normal(loc=self.mean, scale=self.stdev, size=x.shape)
            self.activation = x + noise
        return self.activation
    
class DropoutLayer(IdentityActivation):
    def __init__(self, predecessor, probability=0.5):
        self.predecessor = predecessor
        self.input_size = self.predecessor.output_size        
        self.output_size = self.input_size
        self.activation = np.zeros(self.output_size)
        self.gradient = np.zeros(self.output_size)
        self.init_weights()
        self.require_gradient = False
        # Noise
        self.probability = probability
    def forward(self, x, evaluate=False):
        if evaluate:
            self.activation = x
        else:
            dropout = np.random.choice([0, 1], size=x.shape, p=[self.probability, 1 - self.probability])
            self.activation = (x * dropout)/(1 - self.probability)
        return self.activation
    
class RollLayer(IdentityActivation):
    def __init__(self, predecessor, list_of_offsets=(-1, 0, 1)):
        self.predecessor = predecessor
        self.input_size = self.predecessor.output_size        
        self.output_size = self.input_size
        self.activation = np.zeros(self.output_size)
        self.gradient = np.zeros(self.output_size)
        self.init_weights()
        self.require_gradient = False
        # Noise
        self.list_of_offsets = list_of_offsets
    def forward(self, x, evaluate=False):
        if evaluate:
            self.activation = x
        else:
            offset = np.random.choice(self.list_of_offsets, 1)
            self.activation = np.roll(x, offset, axis=1)
        return self.activation
    # Probably requires a backprop if it isn't at input layer

Implementing batchnorm is a bit trickier. [This blog post helped.](https://wiseodd.github.io/techblog/2016/07/04/batchnorm/) However you really just need to read the original paper (which I didn't have the time to but should have!)

In [12]:
class BatchNormLayer(IdentityActivation):
    def __init__(self, predecessor, eps=0.01):
        self.predecessor = predecessor
        self.input_size = self.predecessor.output_size
        self.output_size = self.input_size
        self.hidden = self.output_size[1]
        self.activation = np.zeros(self.output_size)
        self.gradient = np.zeros(self.output_size)
        self.init_params()
        self.zero_grad()
        self.require_gradient = True
        # Batchnorm requires a constant for "numerical stability"
        self.eps = eps
        # We need to save mean and variance as constants for backprop
        self.mean = np.zeros((1, self.hidden))
        self.var = np.zeros((1, self.hidden))
        # Also, save the xhat
        self.xhat = np.zeros((1, self.hidden))
        # We also want to keep running means and variances during training
        # These become evaluation statistics
        self.eval_mean = np.zeros((1, self.hidden))
        self.eval_var = np.zeros((1, self.hidden))
    def init_params(self):
        # Initialize gamma (mean) and beta (variance)
        self.gamma = Variable(np.ones((1, self.hidden)))
        self.beta = Variable(np.zeros((1, self.hidden)))
    def zero_grad(self):
        self.gamma.zero_grad()
        self.beta.zero_grad()
    def forward(self, x, evaluate=False):
        if evaluate:
            xhat = (x - self.eval_mean) / np.sqrt(self.eval_var + self.eps)
            self.activation = self.gamma.data * xhat + self.beta.data            
        else:
            # Batch mean and variance
            self.mean = np.mean(x, axis=0)
            self.var = np.var(x, axis=0)
            # Evaluation mean and variance
            self.eval_mean = 0.9*self.eval_mean + 0.1*self.mean
            self.eval_var = 0.9*self.eval_var + 0.1*self.var
            # Calculate xhat and the final normalized activation
            self.xhat = (x - self.mean) / np.sqrt(self.var + self.eps)
            self.activation = self.gamma.data * self.xhat + self.beta.data
        return self.activation
    def backprop(self, delta, y):
        N = delta.shape[0]

        self.gamma.grad += np.sum(delta * self.xhat, axis=0)
        self.beta.grad += np.sum(delta, axis=0)

        x_mean = self.predecessor.activation - self.mean
        inv_var_eps = 1 / np.sqrt(self.var + self.eps)

        d_xhat = delta * self.gamma.data
        d_var = np.sum(d_xhat * x_mean, axis=0) * -0.5 * inv_var_eps**3
        d_mean = np.sum(d_xhat * -inv_var_eps, axis=0) + (d_var * np.mean(-2.0 * x_mean))
        delta = (d_xhat * inv_var_eps) + (d_var * 2 * x_mean / N) + (d_mean / N)

        return delta
    def report_params(self):
        return [self.beta, self.gamma]

# Testing the network on real data <a name="30"></a>
## First, testing that things work <a name="31"></a>

The simplest way to test a new network is to feed it what amounts to a truth table and see if it can recreate it.

Below is a list of four 2-bit combinations, `binary_inputs`, and the outputs for `or`, `and`, `nand`, and `xor`.

In [13]:
binary_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
or_outputs = np.array([[0], [1], [1], [1]])
and_outputs = np.array([[0], [0], [0], [1]])
nand_outputs = np.array([[1], [1], [1], [0]])
xor_outputs = np.array([[0], [1], [1], [0]])

current_outputs = xor_outputs

In [14]:
# New network
BinaryNet = NeuralNetwork()

# A very simple network
BinaryNet.layers.append(InputLayer(input_size=2))
BinaryNet.layers.append(DenseLayer(predecessor=BinaryNet.layers[-1], hidden=2, use_bias=True))
BinaryNet.layers.append(SigmoidActivation(predecessor=BinaryNet.layers[-1]))
BinaryNet.layers.append(SigmoidNLL(predecessor=BinaryNet.layers[-1], hidden=1, use_bias=True))

# The SGD optimizer
BinaryNet.optimizer = SGDOptimizer(BinaryNet.layers)

Let's test the FeedForward on a bunch of zeros.

In [15]:
# We'll try each of the possibilities
for output in [or_outputs, and_outputs, nand_outputs, xor_outputs]:
    # Reset the network before starting
    for layer in BinaryNet.layers:
        layer.init_weights()
    # Train it a lot so that it gets to converge
    for i in range(10000):
        BinaryNet.feed_forward(binary_inputs)
        BinaryNet.back_propagation(output)
        BinaryNet.optimizer.step(0.1)
        BinaryNet.zero_gradients()
    print("Expected:")
    print(output)
    print("Got:")
    print(np.around(BinaryNet.feed_forward(binary_inputs), 3))

Expected:
[[0]
 [1]
 [1]
 [1]]
Got:
[[ 0.002]
 [ 0.999]
 [ 0.999]
 [ 1.   ]]
Expected:
[[0]
 [0]
 [0]
 [1]]
Got:
[[ 0.   ]
 [ 0.001]
 [ 0.001]
 [ 0.999]]
Expected:
[[1]
 [1]
 [1]
 [0]]
Got:
[[ 0.999]
 [ 0.999]
 [ 0.999]
 [ 0.002]]
Expected:
[[0]
 [1]
 [1]
 [0]]
Got:
[[ 0.003]
 [ 0.999]
 [ 0.999]
 [ 0.001]]


## Gradient checking <a name="32"></a>

This may be a chore to set up, but it ends up being useful for debugging. In fact, the code above was not working at first, and gradient checking was able to determine that neither the output layers or the dense layers were to blame: by process of elimination, I finally found the typo in the `SigmoidActivation` class.

In [16]:
NNet = NeuralNetwork()

NNet.layers.append(InputLayer(input_size=64))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=32, use_bias=True))
NNet.layers.append(BatchNormLayer(predecessor=NNet.layers[-1]))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=32, use_bias=True))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
NNet.layers.append(SoftmaxCrossEntropy(predecessor=NNet.layers[-1], hidden=10, use_bias=True))

NNet.optimizer = AdaGradOptimizer(NNet.layers)

We need some loss functions. For the time being, these are separate from everything else. Ideally, we'd have a setup like in pytorch and TF where these are optimized and integrated.

I will have to read up on how they did it. Sounds interesting.

In [17]:
def NLLCost(prediction, truth):
    return -np.mean(np.sum(truth*np.log(prediction) + (1.0-truth)*np.log(1.0-prediction), 1))

def CrossEntropy(prediction, truth):
    return -np.mean(np.sum(truth*np.log(prediction), 1))

def accuracy(prediction, labels):
    tests = prediction.argmax(axis=1) == labels
    return(tests.sum() / prediction.shape[0])

Gradient checking is the best way to make sure your back-propagation is working properly.

In [18]:
#
# Early gradient-checking code
#
digits = load_digits(n_class=10, return_X_y=True)
x = digits[0]/16
y = np.eye(10)[digits[1]]

def accuracy(a_out, labels):
    tests = a_out.argmax(axis=1) == labels
    return(tests.sum() / a_out.shape[0])

batch_x = x[0:1]
batch_y = y[0:1]

import copy

layer_number = -1
row = np.random.randint(0, NNet.layers[layer_number].weight.shape[0])
col = np.random.randint(0, NNet.layers[layer_number].weight.shape[1])

# create two copies of your NN
negative_copy = copy.deepcopy(NNet)
positive_copy = copy.deepcopy(NNet)

# nudge them
negative_copy.layers[layer_number].weight.data[row,col] -= 0.0001
positive_copy.layers[layer_number].weight.data[row,col] += 0.0001

# numeric gradient
negative_cost = CrossEntropy(negative_copy.feed_forward(batch_x), batch_y)
positive_cost = CrossEntropy(positive_copy.feed_forward(batch_x), batch_y)
numeric_gradient = (positive_cost - negative_cost)/0.0002
print(numeric_gradient)

NNet.feed_forward(batch_x)

# gradient check
# backprop first
NNet.back_propagation(batch_y)
print(NNet.layers[layer_number].weight.grad[row,col])
NNet.zero_gradients()

0.0
0.0


## Testing the model on the mini-MNIST dataset <a name="33"></a>

Here is a complete run on scikit's mini-MNIST dataset.

In [19]:
lr = 0.1
digits = load_digits(n_class=10, return_X_y=True)
x = digits[0]/16
y = np.eye(10)[digits[1]]
batch_size = 10
batch_pos = list(range(0, digits[0].data.shape[0] - 1, batch_size))
batch_amount = len(batch_pos)

epochs = 10
for ep in range(1, epochs+1):
    batch_num = 1
    for b in batch_pos:
        batch_x = x[b:b+batch_size]
        batch_y = y[b:b+batch_size]
        NNet.feed_forward(batch_x)
        NNet.back_propagation(batch_y)
        NNet.optimizer.step(lr/batch_size)
        NNet.zero_gradients()
    predicted = NNet.feed_forward(x)
    print("epoch {:3d}, lr {:7.5f}, loss {:6.2f}, accuracy {:6.2f}%".format(ep, lr, 
        NLLCost(predicted, y), 100 * accuracy(predicted, digits[1])))

epoch   1, lr 0.10000, loss   1.05, accuracy  87.87%
epoch   2, lr 0.10000, loss   0.54, accuracy  94.88%
epoch   3, lr 0.10000, loss   0.38, accuracy  95.83%
epoch   4, lr 0.10000, loss   0.31, accuracy  96.49%
epoch   5, lr 0.10000, loss   0.27, accuracy  96.66%
epoch   6, lr 0.10000, loss   0.24, accuracy  96.83%
epoch   7, lr 0.10000, loss   0.22, accuracy  96.83%
epoch   8, lr 0.10000, loss   0.21, accuracy  97.11%
epoch   9, lr 0.10000, loss   0.19, accuracy  97.16%
epoch  10, lr 0.10000, loss   0.18, accuracy  97.22%


## Testing the model on the full MNIST dataset <a name="34"></a>

This will download the full MNIST dataset. Uncomment it to run it.

In [20]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

I think we need a better data storage solution than just dumping numpy arrays on the floor and sweeping them into batches.

In [21]:
class DataFeeder:
    def __init__(self, input_dataset, output_dataset):
        self.input = input_dataset
        self.output = output_dataset
        assert self.input.shape[0] == self.output.shape[0]
        self.number_of_rows = self.input.shape[0]
    def shuffle(self):
        p = numpy.random.permutation(self.number_of_rows)
        self.input = self.input[p]
        self.output = self.output[p]

In [25]:
from tqdm import tqdm

lr = 0.01

# training set
train_x = mnist['data'][:60000, :]/256
train_labels = mnist['target'][:60000].astype("int")
train_y = np.eye(10)[train_labels]
# validation set
valid_x = mnist['data'][60000:, :]/256
valid_labels = mnist['target'][60000:].astype("int")
valid_y = np.eye(10)[valid_labels]

epochs = 100
batch_size = 100
batch_pos = list(range(0, train_x.data.shape[0] - 1, batch_size))
batch_amount = len(batch_pos)

NNet = NeuralNetwork()

NNet.layers.append(InputLayer(input_size=784))
NNet.layers.append(RollLayer(predecessor=NNet.layers[-1], list_of_offsets=(-56, -28, -2, -1, 0, 1, 2, 28, 56)))
NNet.layers.append(GaussianNoise(predecessor=NNet.layers[-1]))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=256, use_bias=False, positive_params=False))
NNet.layers.append(BatchNormLayer(predecessor=NNet.layers[-1]))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=256, use_bias=False, positive_params=False))
NNet.layers.append(BatchNormLayer(predecessor=NNet.layers[-1]))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=256, use_bias=False, positive_params=False))
NNet.layers.append(BatchNormLayer(predecessor=NNet.layers[-1]))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
NNet.layers.append(DenseLayer(predecessor=NNet.layers[-1], hidden=256, use_bias=False, positive_params=False))
NNet.layers.append(BatchNormLayer(predecessor=NNet.layers[-1]))
NNet.layers.append(ReLUActivation(predecessor=NNet.layers[-1]))
#NNet.layers.append(DropoutLayer(predecessor=NNet.layers[-1], probability=0.5))
NNet.layers.append(SoftmaxCrossEntropy(predecessor=NNet.layers[-1], hidden=10, use_bias=True))

NNet.optimizer = AdamOptimizer(NNet.layers)

for ep in range(1, epochs+1):
    batch_num = 1
    for b in batch_pos:
        batch_x = train_x[b:b+batch_size]
        batch_y = train_y[b:b+batch_size]
        batch_labels = train_labels[b:b+batch_size]
        NNet.feed_forward(batch_x)
        NNet.back_propagation(batch_y)
        NNet.optimizer.step(lr/batch_size)
        NNet.zero_gradients()
    train_predicted = NNet.evaluate(train_x)
    valid_predicted = NNet.evaluate(valid_x)
    print("Epoch {:3d} stats: lr {:7.5f}, batch size {:3d}, validation loss {:6.2f}".format(ep, lr, batch_size, 
        CrossEntropy(valid_predicted, valid_y)))
    print("  training accuracy {:6.2f}%, validation accuracy {:6.2f}%".format(
        100 * accuracy(train_predicted, train_labels), 100 * accuracy(valid_predicted, valid_labels)))
    #lr *= 0.8
    p = np.random.permutation(train_x.shape[0])
    train_x = train_x[p,:]
    train_labels = train_labels[p]
    train_y = train_y[p,:]

Epoch   1 stats: lr 0.01000, batch size 100, validation loss   2.27
  training accuracy  12.73%, validation accuracy  12.85%
Epoch   2 stats: lr 0.01000, batch size 100, validation loss   0.38
  training accuracy  90.04%, validation accuracy  90.59%
Epoch   3 stats: lr 0.01000, batch size 100, validation loss   0.24
  training accuracy  93.42%, validation accuracy  93.90%
Epoch   4 stats: lr 0.01000, batch size 100, validation loss   0.19
  training accuracy  94.72%, validation accuracy  94.98%
Epoch   5 stats: lr 0.01000, batch size 100, validation loss   0.16
  training accuracy  95.56%, validation accuracy  95.78%
Epoch   6 stats: lr 0.01000, batch size 100, validation loss   0.13
  training accuracy  96.21%, validation accuracy  96.40%
Epoch   7 stats: lr 0.01000, batch size 100, validation loss   0.12
  training accuracy  96.59%, validation accuracy  96.56%
Epoch   8 stats: lr 0.01000, batch size 100, validation loss   0.11
  training accuracy  96.91%, validation accuracy  96.99%


Epoch  67 stats: lr 0.01000, batch size 100, validation loss   0.03
  training accuracy  99.66%, validation accuracy  98.87%
Epoch  68 stats: lr 0.01000, batch size 100, validation loss   0.04
  training accuracy  99.66%, validation accuracy  98.79%
Epoch  69 stats: lr 0.01000, batch size 100, validation loss   0.03
  training accuracy  99.66%, validation accuracy  98.86%
Epoch  70 stats: lr 0.01000, batch size 100, validation loss   0.04
  training accuracy  99.65%, validation accuracy  98.85%
Epoch  71 stats: lr 0.01000, batch size 100, validation loss   0.04
  training accuracy  99.67%, validation accuracy  98.82%
Epoch  72 stats: lr 0.01000, batch size 100, validation loss   0.04
  training accuracy  99.70%, validation accuracy  98.82%
Epoch  73 stats: lr 0.01000, batch size 100, validation loss   0.04
  training accuracy  99.70%, validation accuracy  98.83%
Epoch  74 stats: lr 0.01000, batch size 100, validation loss   0.03
  training accuracy  99.69%, validation accuracy  98.89%
